# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
!pip install python-dotenv

In [6]:
!pip install openai


In [7]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv('/content/.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print("✅ OPENAI Key loaded:", OPENAI_API_KEY is not None)

✅ OPENAI Key loaded: True


In [10]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [11]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [12]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [13]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, of course! *chuckles*


In [14]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [15]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry,  but I don't have access to that information. Can I assist you with anything else?


In [16]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa! Is there anything else you would like to know or talk about?


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [17]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [19]:
!pip install jupyter_bokeh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [22]:
import panel as pn  # GUI
pn.extension()

panels = []  # collect display

# Dictionary to keep track of the user's order details
order_details = {
    'name': '',
    'order': '',
    'size': '',
    'toppings': [],
    'delivery': '',
    'address': '',
    'payment': ''
}

# Define system message (OrderBot's behavior)
context = [{'role': 'system', 'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant.
You first greet the customer, then collect the order,
and then ask if it's a pickup or delivery.
You wait to collect the entire order, then summarize it and check for a final
time if the customer wants to add anything else.
If it's a delivery, you ask for an address.
Finally you collect the payment.
Make sure to clarify all options, extras and sizes to uniquely
identify the item from the menu.
You respond in a short, very conversational friendly style.
The menu includes
pepperoni pizza 12.95, 10.00, 7.00
cheese pizza 10.95, 9.25, 6.50
eggplant pizza 11.95, 9.75, 6.75
fries 4.50, 3.50
greek salad 7.25
Toppings:
extra cheese 2.00,
mushrooms 1.50
sausage 3.00
canadian bacon 3.50
AI sauce 1.50
peppers 1.00
Drinks:
coke 3.00, 2.00, 1.00
sprite 3.00, 2.00, 1.00
bottled water 5.00
"""}]

# Define a function to collect user input and respond
def collect_messages(user_input):
    # Step 1: Collect the user's name if not yet collected
    if order_details['name'] == '':
        order_details['name'] = user_input
        return f"Your name is {order_details['name']}. What would you like to order?"

    # Step 2: Collect the pizza order if not yet collected
    if order_details['order'] == '':
        order_details['order'] = user_input
        return f"Great choice, {order_details['name']}! You ordered a {order_details['order']}. What size would you like? (Small, Medium, Large)"

    # Step 3: Collect the pizza size if not yet chosen
    if order_details['size'] == '':
        order_details['size'] = user_input
        return f"Got it! You chose a {order_details['size']} {order_details['order']}. Would you like to add any toppings? (e.g., extra cheese, mushrooms, sausage)"

    # Step 4: Collect toppings if not yet chosen
    if not order_details['toppings']:
        order_details['toppings'] = user_input.split(',')  # Split the toppings by commas
        return f"Added the following toppings: {', '.join(order_details['toppings'])}. Would you like it for pickup or delivery?"

    # Step 5: Collect pickup or delivery option
    if order_details['delivery'] == '':
        order_details['delivery'] = user_input
        if order_details['delivery'].lower() == 'delivery':
            return f"Got it! {order_details['order']} will be delivered. Please provide your address."
        else:
            return f"Got it! {order_details['order']} will be ready for pickup. Is there anything else you'd like to add?"

    # Step 6: Collect delivery address if chosen delivery
    if order_details['delivery'].lower() == 'delivery' and order_details['address'] == '':
        order_details['address'] = user_input
        return f"Thanks for the address! {order_details['order']} will be delivered to {order_details['address']}. Would you like to add a drink or anything else?"

    # Step 7: Collect payment info if necessary
    if order_details['payment'] == '':
        order_details['payment'] = user_input
        return f"Thank you for your order! {order_details['order']} will be processed. Your total is {calculate_total()}."

    # Step 8: Final check
    return "Thank you for your order! Is there anything else you'd like to add?"

# Function to calculate the total price based on the order
def calculate_total():
    prices = {
        'pepperoni pizza': {'small': 7.00, 'medium': 10.00, 'large': 12.95},
        'cheese pizza': {'small': 6.50, 'medium': 9.25, 'large': 10.95},
        'eggplant pizza': {'small': 6.75, 'medium': 9.75, 'large': 11.95},
        'fries': {'small': 3.50, 'large': 4.50},
        'greek salad': 7.25,
    }

    # Get base price for the pizza
    base_price = prices[order_details['order'].lower()][order_details['size'].lower()]

    # Calculate toppings cost
    toppings_cost = 0
    for topping in order_details['toppings']:
        if topping.strip().lower() == 'extra cheese':
            toppings_cost += 2.00
        elif topping.strip().lower() == 'mushrooms':
            toppings_cost += 1.50
        elif topping.strip().lower() == 'sausage':
            toppings_cost += 3.00
        elif topping.strip().lower() == 'canadian bacon':
            toppings_cost += 3.50
        elif topping.strip().lower() == 'ai sauce':
            toppings_cost += 1.50
        elif topping.strip().lower() == 'peppers':
            toppings_cost += 1.00

    # Add drink cost (if any)
    drink_cost = 0
    if 'coke' in order_details['toppings']:
        drink_cost += 3.00
    elif 'sprite' in order_details['toppings']:
        drink_cost += 3.00
    elif 'bottled water' in order_details['toppings']:
        drink_cost += 5.00

    total = base_price + toppings_cost + drink_cost
    return round(total, 2)

# Widgets for user input and chatbot responses
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here...')
button_conversation = pn.widgets.Button(name="Chat!")

# Bind the user input to the conversation function
interactive_conversation = pn.bind(collect_messages, inp)

# Dashboard layout
dashboard = pn.Column(
    inp,
    button_conversation,
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


Column
    [0] TextInput(placeholder='Enter text here...', value='Hi')
    [1] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Markdown, defer_load=False, height=300, loading_indicator=True)

In [23]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": [
    {
      "type": "pepperoni pizza",
      "size": "large",
      "price": 12.95
    }
  ],
  "toppings": [
    {
      "type": "extra cheese",
      "price": 2.00
    },
    {
      "type": "mushrooms",
      "price": 1.50
    }
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium",
      "price": 2.00
    }
  ],
  "sides": [],
  "total price": 18.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [31]:
import panel as pn  # GUI
pn.extension()

# Initialize the order_details dictionary with empty values for all fields
order_details = {
    'name': '',
    'order': '',
    'size': '',
    'toppings': [],
    'delivery': '',
    'address': '',
    'departure_date': '',  # Initialize 'departure_date'
    'return_date': '',     # Initialize 'return_date'
    'flight_class': '',    # Initialize 'flight_class'
    'payment': ''
}

# Store conversation history
conversation_history = []

# Function to collect user messages and process travel details
def collect_travel_details(user_input):
    # Append the user's input to the conversation history
    conversation_history.append(f"You: {user_input}")

    # Step 1: Collect the user's name if not yet collected
    if order_details['name'] == '':
        order_details['name'] = user_input
        response = f"Hello, {order_details['name']}! Where would you like to travel?"

    # Step 2: Collect the destination if not yet collected
    elif order_details['order'] == '':
        order_details['order'] = user_input
        response = f"Great! You're going to {order_details['order']}. What is your departure date?"

    # Step 3: Collect the departure date
    elif order_details['departure_date'] == '':
        order_details['departure_date'] = user_input
        response = f"Got it! You're departing on {order_details['departure_date']}. What is your return date?"

    # Step 4: Collect the return date
    elif order_details['return_date'] == '':
        order_details['return_date'] = user_input
        response = f"Your return date is set for {order_details['return_date']}. What flight class would you prefer? (Economy, Business, First)"

    # Step 5: Collect the flight class
    elif order_details['flight_class'] == '':
        order_details['flight_class'] = user_input
        response = f"Excellent! You chose {order_details['flight_class']} class. Would you like to proceed with payment?"

    # Step 6: Collect payment
    elif order_details['payment'] == '':
        order_details['payment'] = user_input
        response = f"Thank you for booking your flight to {order_details['order']}! Your payment is {order_details['payment']}."

    else:
        response = "Thank you for your booking! Is there anything else you'd like to add?"

    # Append the bot's response to the conversation history
    conversation_history.append(f"OrderBot: {response}")

    # Update the conversation panel with the full conversation history
    conversation_text = "\n".join(conversation_history)
    return conversation_text

# Widgets for user input and chatbot responses
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here...')
button_conversation = pn.widgets.Button(name="Send")

# Bind the user input to the conversation function
interactive_conversation = pn.bind(collect_travel_details, inp)

# Display the conversation history in the panel
conversation_display = pn.pane.Markdown("", width=600, height=400)

# Update the conversation display when the user sends a message
def update_conversation(event):
    conversation_text = collect_travel_details(inp.value)
    conversation_display.object = conversation_text

# Attach the update function to the button
button_conversation.on_click(update_conversation)

# Dashboard layout
dashboard = pn.Column(
    inp,
    button_conversation,
    conversation_display
)

dashboard


Column
    [0] TextInput(placeholder='Enter text here...', value='Hi')
    [1] Button(name='Send')
    [2] Markdown(str, height=400, sizing_mode='fixed', width=600)

Flight Booking Bot Report
Objective:

The goal of this code is to create an OrderBot for booking flight tickets interactively. The bot collects the following information from the user:

Name
Destination
Departure date
Return date
Flight class (Economy, Business, First)
Payment details to confirm the booking
Main Features:

Interactive Information Collection:
The bot greets the user and asks for their name.
It then asks for the destination, departure date, return date, and flight class.
Finally, it collects payment information to confirm the booking.
Conversation History:
All user inputs and bot responses are stored in conversation_history and displayed continuously on the dashboard.
Total Calculation:
The bot calculates the total cost based on the user's flight class choice using the calculate_total() function.
Continuous User Interaction:
The bot responds immediately after each user input and updates the conversation history in real-time.
Outcome:

The bot works smoothly and collects all necessary information to complete the flight booking.
The user is shown a real-time conversation and the total cost of their booking.
Key Observations:

User-Friendly: The process is simple and easy to follow.
Flexibility: The bot is flexible in handling different types of user inputs (destination, dates, etc.).
Real-Time Updates: The dashboard displays the entire conversation, providing a seamless experience.
Conclusion:

The flight booking bot is functional and provides an interactive, smooth booking experience. It can be further improved by adding input validation and additional features, such as available flight dates based on the destination.